In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI
import time
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime
from utils import request_wapi,get_forecast,create_df,send_message,get_date

In [112]:
query = 'Medellin'
api_key = API_KEY_WAPI

url = 'http://api.weatherapi.com/v1/forecast.json?key={}&q={}&days=1&aqi=no&alerts=no'.format(api_key,query)

In [113]:
response = requests.get(url).json()

In [114]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [115]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [116]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [117]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-06-20'

In [118]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [119]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Clear'

In [120]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

12.4

In [121]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [122]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

## Dataframe

In [123]:
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    rain_prov = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,temp,rain,rain_prov

In [124]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'):

    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [125]:
datos

[('2023-06-20', '00', 'Clear', 12.4, 0, 0),
 ('2023-06-20', '01', 'Clear', 12.1, 0, 0),
 ('2023-06-20', '02', 'Partly cloudy', 12.0, 0, 0),
 ('2023-06-20', '03', 'Partly cloudy', 11.8, 0, 0),
 ('2023-06-20', '04', 'Partly cloudy', 11.7, 0, 0),
 ('2023-06-20', '05', 'Partly cloudy', 11.6, 0, 0),
 ('2023-06-20', '06', 'Partly cloudy', 11.5, 0, 0),
 ('2023-06-20', '07', 'Sunny', 14.9, 0, 0),
 ('2023-06-20', '08', 'Sunny', 18.8, 0, 0),
 ('2023-06-20', '09', 'Sunny', 21.2, 0, 0),
 ('2023-06-20', '10', 'Sunny', 23.1, 0, 0),
 ('2023-06-20', '11', 'Sunny', 25.6, 0, 0),
 ('2023-06-20', '12', 'Sunny', 28.4, 0, 0),
 ('2023-06-20', '13', 'Sunny', 29.0, 0, 0),
 ('2023-06-20', '14', 'Sunny', 28.9, 0, 0),
 ('2023-06-20', '15', 'Sunny', 28.3, 0, 0),
 ('2023-06-20', '16', 'Sunny', 26.1, 0, 0),
 ('2023-06-20', '17', 'Sunny', 24.4, 0, 0),
 ('2023-06-20', '18', 'Sunny', 20.2, 0, 0),
 ('2023-06-20', '19', 'Clear', 17.1, 0, 0),
 ('2023-06-20', '20', 'Clear', 14.2, 0, 0),
 ('2023-06-20', '21', 'Partly cloudy

In [126]:
col = ['Fecha','Hora','Condicion','Temp','Rain','Rain_prov']
df = pd.DataFrame(datos,columns=col)

In [127]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-20,00,Clear,12.4,0,0
1,2023-06-20,01,Clear,12.1,0,0
2,2023-06-20,02,Partly cloudy,12.0,0,0
3,2023-06-20,03,Partly cloudy,11.8,0,0
4,2023-06-20,04,Partly cloudy,11.7,0,0
5,2023-06-20,05,Partly cloudy,11.6,0,0
6,2023-06-20,06,Partly cloudy,11.5,0,0
7,2023-06-20,07,Sunny,14.9,0,0
8,2023-06-20,08,Sunny,18.8,0,0
9,2023-06-20,09,Sunny,21.2,0,0


In [128]:
df = df[(df['Rain'] == 1)]
df_redu = df[['Hora','Condicion']]
df_redu.set_index('Hora',inplace = True)

In [129]:

if df_redu.empty:
    print('no hay plan arrunchis, no va a llover :C')
else:
    print('wuju')

no hay plan arrunchis, no va a llover :C


In [130]:
df_redu

,Condicion
Hora,


In [131]:
PHONE_NUMBER

'+14027874717'

## Mensaje SMS desde twilio

In [132]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client


In [133]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

In [134]:
'Hoy va a llover, abrígate bien :) \n\n\n ' + str(df_redu)
k = '+573154472305'
j = '+573185953816'

In [111]:


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure

client = Client(account_sid, auth_token)
if df_redu.empty:
    template = 'Hoy no hay arrunchis, no va a llover :c'
else:
    template = 'Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_redu)

message = client.messages \
    .create(
        
        body=template,
        from_=PHONE_NUMBER,
        to=j
     )

print('Mensaje enviado: ',message.sid, template)



Mensaje enviado:  SM7b55d9ee042e766a52b21522bb649d4c Hoy va a llover en Bogotá, abrígate bien :) 


                  Condicion
Hora                      
14    Patchy rain possible
15    Patchy rain possible
16    Patchy rain possible
18    Patchy rain possible
19    Patchy rain possible
